In [1]:
import torch
from torch import nn, optim
import torch.nn.functional as F
from torch.utils.data import DataLoader, TensorDataset
import numpy as np
import pandas as pd
from sklearn.metrics import mean_squared_error

/opt/conda/lib/python3.8/site-packages/pandas/compat/_optional.py:161: UserWarning: Pandas requires version '2.7.1' or newer of 'numexpr' (version '2.7.0' currently installed).
  warnings.warn(msg, UserWarning)


In [10]:
import json

with open('../data/preprocessed/3_month_retail.json', 'r') as f:
    data = json.load(f)

with open('../data/preprocessed/cluster_store.json','r') as f:
    cluster = json.load(f)
with open('../data/preprocessed/cluster_key_store_num.json', 'r') as f:
    cluster_key_store = json.load(f)

In [11]:
# cluster 번호를 지정해주면, 거기 안의 데이터를 X로 return 해주는 함수를 만들자
def cluster_dataset(cluster_num):
    data_tensor = torch.tensor([])
    for store_id in cluster[cluster_num]:
        sub_tensor = torch.tensor(list(data[str(store_id)].values()))
        data_tensor = torch.cat((data_tensor,sub_tensor),0)
    return data_tensor[:,:-1].t(), data_tensor[:,-1].t()

In [112]:
X_train,X_test = cluster_dataset('0')

In [113]:
X_train[0].shape

torch.Size([7848])

In [115]:
a1 = X_train[0].reshape(1, 1, 7848)

In [116]:
layer_norm = nn.LayerNorm(7848)

In [117]:
a2 = layer_norm(a1).reshape(1, 1, 18, 436)

In [118]:
layer_3 = nn.Conv2d(1, 1, kernel_size=(4, 4))

In [119]:
a3 = layer_3(a2)
a3.size()

torch.Size([1, 1, 15, 433])

In [120]:
F.max_pool2d(a3, (4,4)).size()

torch.Size([1, 1, 3, 108])

In [76]:
class DSModel0(nn.Module):
    def __init__(self):
        super(DSModel0, self).__init__()
        # self.layer1 = torch.nn.Sequential(
        #     torch.nn.Conv1d(1098, 256, kernel_size=2),
        #     torch.nn.ReLU(),
        #     torch.nn.MaxPool1d(kernel_size=2, stride=1)
        # )
        self.fc1 = nn.Linear(7830,1024)
        self.fc2 = nn.Linear(1024,256)
        self.fc3 = nn.Linear(256,64)
        self.fc4 = nn.Linear(64,8)
        self.fc5 = nn.Linear(8,1)
    
    def forward(self, x):
        # x = F.relu(self.layer1(x))
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.relu(self.fc3(x))
        x = F.relu(self.fc4(x))
        x = F.relu(self.fc5(x))
        return x

class DSModel1(nn.Module):
    def __init__(self):
        super(DSModel1, self).__init__()
        # self.layer1 = torch.nn.Sequential(
        #     torch.nn.Conv1d(1098, 256, kernel_size=2),
        #     torch.nn.ReLU(),
        #     torch.nn.MaxPool1d(kernel_size=2, stride=1)
        # )
        self.fc1 = nn.Linear(1098,256)
        self.fc2 = nn.Linear(256,64)
        self.fc3 = nn.Linear(64,8)
        self.fc4 = nn.Linear(8,1)
    
    def forward(self, x):
        # x = F.relu(self.layer1(x))
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.relu(self.fc3(x))
        x = F.relu(self.fc4(x))
        return x

class DSModel2(nn.Module):
    def __init__(self):
        super(DSModel2, self).__init__()
        # self.layer1 = torch.nn.Sequential(
        #     torch.nn.Conv1d(1098, 256, kernel_size=2),
        #     torch.nn.ReLU(),
        #     torch.nn.MaxPool1d(kernel_size=2, stride=1)
        # )
        self.fc1 = nn.Linear(18,8)
        self.fc2 = nn.Linear(8,1)
    
    def forward(self, x):
        # x = F.relu(self.layer1(x))
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        return x

class DSModel3(nn.Module):
    def __init__(self):
        super(DSModel3, self).__init__()
        # self.layer1 = torch.nn.Sequential(
        #     torch.nn.Conv1d(1098, 256, kernel_size=2),
        #     torch.nn.ReLU(),
        #     torch.nn.MaxPool1d(kernel_size=2, stride=1)
        # )
        self.fc1 = nn.Linear(2610,256)
        self.fc2 = nn.Linear(256,64)
        self.fc3 = nn.Linear(64,8)
        self.fc4 = nn.Linear(8,1)
    
    def forward(self, x):
        # x = F.relu(self.layer1(x))
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.relu(self.fc3(x))
        x = F.relu(self.fc4(x))
        return x

In [78]:
# define store_nums
store_list = [*range(643)]

In [79]:
result_tensor = torch.tensor([]).to(device)
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

for store_id in store_list:
    #상점이 어느 cluster에 해당하는지
    cluster_num = cluster_key_store[str(store_id)]
    # X = cluster_dataset(cluster_num)
    # X_train = X[:27]
    # X_test = X[27:]
    X_train, X_test = cluster_dataset(cluster_num)
    # X_train, X_test size : (28, ~)
    X_train = X_train.to(device)
    X_test = X_test.to(device)
    #product
    loss_ = {}
    for item in data[str(store_id)].keys():
        y = torch.tensor(data[str(store_id)][item][1:]).float()
        y_train = y 
        y_train = y_train.to(device)
        dataset_train = TensorDataset(X_train, y_train)
        train_dataloader = DataLoader(dataset_train, batch_size=1, shuffle=False)

        if cluster_num == '0':
            model = DSModel0()
        elif cluster_num == '1':
            model = DSModel1()
        elif cluster_num == '2':
            model = DSModel2()
        else:
            model = DSModel3()

        model = model.to(device)
        criterion = nn.MSELoss()
        optimizer = optim.Adam(model.parameters(), lr = 0.001, weight_decay=1e-7)

        # item_loss = 0.0
        for epoch in range(20):
            # running_loss = 0.0
            for i, data_input in enumerate(train_dataloader,0):
                inputs, value = data_input
                optimizer.zero_grad()
                outputs = model(inputs).flatten()
                loss = criterion(outputs, value)
                loss.backward()
                optimizer.step()

                # running_loss += loss.item()

        #item inference
        with torch.no_grad():
            y_pred = model(X_test)
        result_tensor = torch.cat((result_tensor, y_pred),0)
        



In [73]:
# result_tensor을 dataframe으로 전환
import pandas as pd 
import numpy as np

result_tensor_df = pd.DataFrame(result_tensor.detach().cpu().numpy())

#round시키고 int로 바꿔야함
result_tensor_df = result_tensor_df.round(0).astype(int)

In [74]:
result_tensor_df.to_csv('first_draft2.csv')

In [2]:
import pandas as pd
import numpy as np

/opt/conda/lib/python3.8/site-packages/pandas/compat/_optional.py:161: UserWarning: Pandas requires version '2.7.1' or newer of 'numexpr' (version '2.7.0' currently installed).
  warnings.warn(msg, UserWarning)


In [3]:
a = pd.read_csv('first_draft.csv')
b = pd.read_csv('first_draft2.csv')

In [8]:
a['0'].tolist()

[0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 2,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,


In [10]:
new = a['0'].tolist() + b['0'].tolist()

In [11]:
len(new)

11556

In [13]:
sum(new)

882